### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [24]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

from scipy.stats import norm

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [25]:
hyperparameterspace = {
    'x0': ["interval", -5, 10],
    'x1': ["interval", -5, 10],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [26]:


BUDGETS = [1 + i * 52 for i in range(19)]

### Optimization

In [28]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    sum = 0
    sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    return sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    
    return len(hyperparameterspace) * 10 + sum

def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1])


BUDGET = 400
INIT_POINTS = 5

print("INITIAL POINTS:")

reset_seeds()

optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, INIT_POINTS, 0, 0.5, init_points=INIT_POINTS, alternative=2)
points = optimizer.fit()

index_refine = 0

distances = [math.dist(
    points[index_refine].get_coordinates(), x.get_coordinates()) for x in points]
highest_distance = max(distances)

# find smallest distance (replace smallest distance zero with highest distance)
corners = [[], []]
for key in hyperparameterspace.keys():
    corners[0].append(hyperparameterspace.get(key)[1])
    corners[1].append(hyperparameterspace.get(key)[2])

smallest_dist = math.dist(corners[0], corners[1])
for i in range(len(points)):

    if distances[i] == 0:
        distances[i] = highest_distance

    if i != index_refine and distances[i] < smallest_dist and points[i].get_level() <= points[index_refine].get_level():
        smallest_dist = distances[i]

smallest_dist = (highest_distance + smallest_dist) / ((points[index_refine].get_level()+2)*2)
#smallest_dist = 1# (highest_distance + smallest_dist) / ((points[index_refine].get_level()+2)*4)
# smallest_dist = (highest_distance + smallest_dist) / 2

coordinates = []


lower = max([points[index_refine].get_coordinates()[0]-smallest_dist, hyperparameterspace.get('x0')[1]])
lower = min([lower, hyperparameterspace.get('x0')[2]])

upper = min([points[index_refine].get_coordinates()[0]+smallest_dist, hyperparameterspace.get('x0')[2]])
upper = max([upper, hyperparameterspace.get('x0')[1]])



x_axis = np.arange(-5, 11, 0.1)

mean_x, std_x = (upper+lower)/2, (upper-lower)/2
    

lower = max([points[index_refine].get_coordinates()[1]-smallest_dist, hyperparameterspace.get('x1')[1]])
lower = min([lower, hyperparameterspace.get('x1')[2]])

upper = min([points[index_refine].get_coordinates()[1]+smallest_dist, hyperparameterspace.get('x1')[2]])
upper = max([upper, hyperparameterspace.get('x1')[1]])



y_axis = np.arange(-5, 11, 0.1)

mean_y, std_y = (upper+lower)/2, (upper-lower)/2
    
Z = np.zeros((len(x_axis), len(y_axis)))

for x in range(len(x_axis)):
    for y in range(len(y_axis)):
        Z[x][y] = norm.pdf(x_axis[x], mean_x, std_x) * norm.pdf(y_axis[y], mean_y, std_y)


# plt.plot(x_axis, norm.pdf(x_axis, mean, sd))



x_values = []
y_values = []
z_values = [1, 1, 1, 1, 1]

for i in range(len(points)):    
    x_values.append(points[i].get_coordinates()[0])
    y_values.append(points[i].get_coordinates()[1])


fig = plt.figure()
ax = plt.axes()
surface = plt.scatter(x_values, y_values, c="red")
plt.scatter([4.089162485176937], [4.1020260046125685], c="blue", marker="x")

cont = plt.contourf(x_axis, y_axis, Z, alpha=0.45, cmap="plasma", levels=20)

# circle1 = plt.Circle((points[0].get_coordinates()[0], points[0].get_coordinates()[1]), smallest_dist, color='blue', alpha=0.16)

# Add the patch to the Axes
#ax.add_patch(circle1)

ax.set_xlim([-5, 10])
ax.set_ylim([-5, 10])


plt.colorbar(cont)
plt.gca().set_aspect("equal")

plt.savefig("./Alternative_3.pgf",bbox_inches='tight' )

plt.show()


INITIAL POINTS:


/tmp/ipykernel_14532/2082996060.py:125: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
